# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather = "cities_weather2.csv"

weatherDF = pd.read_csv(weather)

weatherDF.head()

,Unnamed: 0,City,Country,Temp,Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Date
0,0,norman wells,CA,30.20,92,75,2.24,65.28,-126.83,1602427127
1,1,yellowknife,CA,35.01,100,90,9.17,62.46,-114.35,1602426964
2,2,labuhan,ID,82.71,75,56,6.73,-6.88,112.21,1602427127
3,3,punta arenas,CL,51.80,62,20,38.03,-53.15,-70.92,1602427127
4,4,barrow,US,24.80,68,90,19.46,71.29,-156.79,1602427127


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weatherDF[["Latitude", "Longitude"]]
humidity = weatherDF["Humidity"]

In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
perfect_weather = weatherDF.loc[(weatherDF['Temp'] < 90) & (weatherDF["Temp"] >= 70) \
& (weatherDF["Wind Speed"] < 15) & (weatherDF["Cloudiness"] <5)].dropna()

perfect_weather.head()

,Unnamed: 0,City,Country,Temp,Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Date
22,22,barkly west,ZA,82.40,23,0,5.82,-28.54,24.52,1602427129
40,40,najran,SA,86.00,23,0,4.70,17.49,44.13,1602427033
42,42,darbhanga,IN,79.14,59,0,2.86,26.17,85.90,1602427131
49,49,concepcion del uruguay,AR,71.01,37,2,5.99,-32.48,-58.24,1602427132
72,72,salalah,OM,82.40,69,0,9.17,17.02,54.09,1602427117


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = perfect_weather

hotel_df.drop(['Unnamed: 0'], axis='columns',inplace=True)

hotel_df.head()

,City,Country,Temp,Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Date
22,barkly west,ZA,82.40,23,0,5.82,-28.54,24.52,1602427129
40,najran,SA,86.00,23,0,4.70,17.49,44.13,1602427033
42,darbhanga,IN,79.14,59,0,2.86,26.17,85.90,1602427131
49,concepcion del uruguay,AR,71.01,37,2,5.99,-32.48,-58.24,1602427132
72,salalah,OM,82.40,69,0,9.17,17.02,54.09,1602427117


In [10]:
hotel_df.reset_index(drop=True).head()

,City,Country,Temp,Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Date
0,barkly west,ZA,82.40,23,0,5.82,-28.54,24.52,1602427129
1,najran,SA,86.00,23,0,4.70,17.49,44.13,1602427033
2,darbhanga,IN,79.14,59,0,2.86,26.17,85.90,1602427131
3,concepcion del uruguay,AR,71.01,37,2,5.99,-32.48,-58.24,1602427132
4,salalah,OM,82.40,69,0,9.17,17.02,54.09,1602427117


In [13]:
hotel_df['Hotel Name'] = '' 

#hotel_df.head()

In [18]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# geocoordinates
target_radius = 5000
target_search = "hotel"
target_type = "lodging"

# set up a parameters dictionary
params = {
    "radius": target_radius,
    "keyword": target_search,
    "type": target_type,
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get type from df
    lat = row['Latitude']
    lon = row['Longitude']
    city = row['City']
    params["location"] = f'{lat},{lon}'

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {city} hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    
    except:
        print("Missing field/result for {city}... skipping.")
        
    print("------------")

Closest barkly west hotel is Die Olyfhuis.
------------
Missing field/result for {city}... skipping.
------------
Closest darbhanga hotel is G V HERITAGE HOTEL ..
------------
Closest concepcion del uruguay hotel is Hotel Gran Litoral.
------------
Closest salalah hotel is Grand flora hotel salalah.
------------
Closest rikitea hotel is Pension Maro'i.
------------
Missing field/result for {city}... skipping.
------------
Closest simdega hotel is OYO 68608 Hotel Amanlok.
------------
Closest greenville hotel is Hilton Greenville.
------------
Closest cruzeiro do sul hotel is Estrela Palace Hotel.
------------
Missing field/result for {city}... skipping.
------------
Closest buraydah hotel is فندق براند القصيم.
------------
Closest gerash hotel is Sa'adat House.
------------
Closest port keats hotel is TDC Village (Hundred Man Camp).
------------
Closest myitkyina hotel is Palm Spring Resort Hotel Myitkyina.
------------
Missing field/result for {city}... skipping.
------------
Closest 

In [23]:
hotel_df=hotel_df[hotel_df['Hotel Name'].notna()]
hotel_df=hotel_df.rename(columns={"Latitude": "Lat", "Longitude": "Lng"})
hotel_df.head()

,City,Country,Temp,Humidity,Cloudiness,Wind Speed,Lat,Lng,Date,Hotel Name
22,barkly west,ZA,82.40,23,0,5.82,-28.54,24.52,1602427129,Die Olyfhuis
40,najran,SA,86.00,23,0,4.70,17.49,44.13,1602427033,
42,darbhanga,IN,79.14,59,0,2.86,26.17,85.90,1602427131,G V HERITAGE HOTEL .
49,concepcion del uruguay,AR,71.01,37,2,5.99,-32.48,-58.24,1602427132,Hotel Gran Litoral
72,salalah,OM,82.40,69,0,9.17,17.02,54.09,1602427117,Grand flora hotel salalah


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [25]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))